In [ ]:
# Kindly note that some of the files aren't composed into a new data set yet, as they were 
# downloaded using a beatuifulsoup from the Nasa Space Apps Challenge 2024 resources. 
# For more information on the project: https://github.com/Ahmed-Samir11/Cosmic-Analysts

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
from obspy import read
from obspy import UTCDateTime
from torchinfo import summary
from utils import *
from autoencoder_model import *
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Set the plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")



# Explore an example of data

In [ ]:
file_path = r"xb.elyse.00.hhv.2019.066.1.mseed" #File Manually downloaded from the full data archive
try:
    stream = read(file_path)
    print("File successfully read!")
    print(stream)
except Exception as e:
    print(f"Error reading file with ObsPy: {e}")
for tr in stream:
    print("\nTrace Statistics:")
    print(f"Network: {tr.stats.network}")
    print(f"Station: {tr.stats.station}")
    print(f"Channel: {tr.stats.channel}")
    print(f"Location: {tr.stats.location}")
    print(f"Start Time: {tr.stats.starttime}")
    print(f"End Time: {tr.stats.endtime}")
    print(f"Sampling Rate: {tr.stats.sampling_rate} Hz")
    print(f"Number of Samples: {tr.stats.npts}")
    print(f"Data Type: {tr.data.dtype}")
    # If needed, you can also access the raw data
    print(f"Data: {tr.data}")

# Print general metadata
print("\nGeneral Metadata:")
print(f"Number of Traces: {len(stream)}")


In [ ]:
trace = stream[0]
plt.figure(figsize=(10, 6))
plt.plot(trace.times(), trace.data)
plt.xlabel('Time (s)')
plt.ylabel('Velocity')
plt.title('Seismic Trace')
plt.show()

In [ ]:
guided_df = pd.read_csv("Mars_InSight_training_catalog.csv")
guided_df

In [ ]:
row = guided_df.iloc[0]
row

In [ ]:
arrival_time = datetime.strptime(row['time'],'%Y-%m-%dT%H:%M:%S.%f')

In [ ]:
mseed_file = r'C:\Users\ahmed\OneDrive\Desktop\Cosmic Analysts\XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed'
st = read(mseed_file)
print(st)
st[0].stats

In [ ]:
tr = st.traces[0].copy()
tr_times = tr.times()
tr_data = tr.data
starttime = tr.stats.starttime.datetime

# Create a vector for the absolute time
tr_times_dt = []
for tr_val in tr_times:
    tr_times_dt.append(starttime + timedelta(seconds=tr_val))

# Plot the absolute result
fig,ax = plt.subplots(1,1,figsize=(10,3))

# Plot trace
ax.plot(tr_times_dt,tr_data)

# Mark detection
arrival_line = ax.axvline(x=arrival_time, c='red', label='Abs. Arrival')
ax.legend(handles=[arrival_line])

# Make the plot pretty
ax.set_xlim([min(tr_times_dt),max(tr_times_dt)])
ax.set_ylabel('Velocity (m/s)')
ax.set_xlabel('Time (s)')
ax.set_title('XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed', fontweight='bold')

In [ ]:
mseed_file = r'xb.elyse.02.bhv.2022.034.7.mseed'
st = read(mseed_file)
print(st)
st[0].stats

In [ ]:
tr = st.traces[0].copy()
tr_times = tr.times()
tr_data = tr.data
starttime = tr.stats.starttime.datetime
endtime = tr.stats.endtime.datetime
# Create a vector for the absolute time
tr_times_dt = []
for tr_val in tr_times:
    tr_times_dt.append(starttime + timedelta(seconds=tr_val))

# Plot the absolute result
fig,ax = plt.subplots(1,1,figsize=(10,3))

# Plot trace
ax.plot(tr_times_dt,tr_data)

# Mark detection
arrival_line = ax.axvline(x=arrival_time, c='red', label='Abs. Arrival')
ax.legend(handles=[arrival_line])

# Make the plot pretty
ax.set_xlim([min(tr_times_dt),max(tr_times_dt)])
ax.set_ylabel('Velocity (m/s)')
ax.set_xlabel('Time (s)')
ax.set_title('XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed', fontweight='bold')

# Make DataFrame

### Map each location to its meaning and measurements

How files are named?
Mini-SEED data file:  [network].[station].[location].[channel].[year].[doy].[rev].mseed


Further Explanation: \
The code is made up of 3 letters: Instrument code, Band code, and orientation code. 

In [ ]:
instrument_code = {'H': 'High Gain Seismometer',
                   'L': 'Low Gain Seismometer',
                   'M': 'Mass Position Seismometer',
                   'D': 'Pressure',
                   'F': 'Magnetometer',
                   'k': 'Temperature',
                   'W': 'Wind',
                   'Z': 'Sythetized Beam Data',
                   'Y': 'Non-specific Instruments',
                   'E': 'Electric Test Point'
                   }

In [ ]:
band_code = { 'SP' : {
    'E': 100,
    'S': [10, 80],
    'M': [2, 5],
    'L': 1,
    'V': [0.1 , 0.5],
    'U': [0.01, 0.05],
    'R': [1/3600, 0.001]
}, 
'VBB': {
    'H': 100,
    'B': [10, 80],
    'M': [2, 5],
    'L': 1,
    'V': [0.1 , 0.5],
    'U': [0.01, 0.05],
    'R': [1/3600, 0.001]
}
}
# List elements are range, not discrete values

SP: Short Period Seismometer \
VBB: Very Broadband Seismometer \
APPS: Auxiliary Payload Sensor Suite

In [ ]:
# Define the dictionary
loc_id = {
    'SEIS/VBB related data': {
        'Science': { 
            'High G.': {
                'VBB': '00',
                'Replaced SP': '20', # Synthesized SP  (from VBB1, VBB2, VBB3)
                'spare': None, 
                'VBB RMS': '40', # High pass RMS over one second
                'MAX VBB RMS': '45', # Maximum RMS over N seconds
                'spare': '50'
                }, 
            'Low G.': {
                'VBB': '05',
                'Replaced SP': None,
                'spare': None,
                'VBB RMS': None,
                'MAX VBB RMS': None,
                'spare': None
            }
            
        },
        'Engin.': {
            'High G.': {
                'VBB': '10',
                'Replaced SP': None,
                'spare': '30', # spare Ids for possible VBB open loop mode
                'VBB RMS': None,
                'MAX VBB RMS': None,
                'spare': None
                }, 
            'Low G.': {
                'VBB': '15',
                'Replaced SP': None,
                'spare': '35', # spare Ids for possible VBB open loop mode
                'VBB RMS': None,
                'MAX VBB RMS': None,
                'spare': None
            }
        }
    },
    'SEIS/Hybrid': { # Hybrid channels
        'Science': {
            'High G.': {
                'VBB+SP': '55',
                'spare': '60'
                }, 
            'Low G.': {
                'VBB+SP': None,
                'spare': None
            }
        },
        'Engin.': {
            'High G.': {
                'VBB+SP': None,
                'spare': None
                }, 
            'Low G.': {
                'VBB+SP': None,
                'spare': None
            }
        }
    },
    'SEIS/SP related data': {
        'Science': {
            'High G.': {
                'SP': '65',
                'Rotated SP': '75', # On board rotated SP (from SP1, SP2, SP3)
                'Replaced VBB': '80', # Synthesized VBB (from SP1, SP2, SP3)
                'SP RMS': '85', # High pass RMS over one second
                'MAX SP RMS': '90', # Maximum RMS over N seconds
                'spare': '95'
                }, 
            'Low G.': {
                'SP': '70',
                'Rotated SP': None,
                'Replaced VBB': None,
                'SP RMS': None,
                'MAX SP RMS': None,
                'spare': None
            }
        },
        'Engin.': {
            'High G.': {
                'SP': None,
                'Rotated SP': None,
                'Replaced VBB': None,
                'SP RMS': None,
                'MAX SP RMS': None,
                'spare': None
                }, 
            'Low G.': {
                'SP': None,
                'Rotated SP': None,
                'Replaced VBB': None,
                'SP RMS': None,
                'MAX SP RMS': None,
                'spare': None
            }
        }
    },
    'APSS related data': {
        'Science': {
            'High G.': {
                'TWINS proc 1': '00', # Magnetometer, Pressure, temperature (raw data)
                'TWINS Proc 2': '10', # On Earth Processed Data: wind amplitude and direction, atmospheric temperature
                'Rotated MAG': '20', # On board rotated MAG (from mag1, mag2, mag3)
                'MAG RMS': '30', # High pass RMS over one second
                'MAX MAG RMS': '40', # Maximum RMS over N seconds
                'P1 RMS': '50', # High pass RMS over one second
                'P2 RMS': '60',
                'MAX P1 RMS': '70', # Maximum RMS over N seconds
                'MAX P2 RMS': '80',
                'spare': '90'
                }, 
            'Low G.': {
                'TWINS proc 1': None,
                'TWINS Proc 2': None,
                'Rotated MAG': None,
                'MAG RMS': None,
                'MAX MAG RMS': None,
                'P1 RMS': None,
                'P2 RMS': None,
                'MAX P1 RMS': None,
                'MAX P2 RMS': None,
                'spare': None
            }
        },
        'Engin.': {
            'High G.': {
                'TWINS proc 1': None,
                'TWINS Proc 2': None,
                'Rotated MAG': None,
                'MAG RMS': None,
                'MAX MAG RMS': None,
                'P1 RMS': None,
                'P2 RMS': None,
                'MAX P1 RMS': None,
                'MAX P2 RMS': None,
                'spare': None
                }, 
            'Low G.': {
                'TWINS proc 1': None,
                'TWINS Proc 2': None,
                'Rotated MAG': None,
                'MAG RMS': None,
                'MAX MAG RMS': None,
                'P1 RMS': None,
                'P2 RMS': None,
                'MAX P1 RMS': None,
                'MAX P2 RMS': None,
                'spare': None
            }
        }
    }
}

df_list = []
for category, data in loc_id.items():
    for sub_category, values in data.items():
        df = pd.DataFrame(values).T
        df.insert(0, 'Category', category)
        df.insert(1, 'Sub-Category', sub_category)
        df_list.append(df)

#df_combined = pd.concat(df_list, ignore_index=True)
#df_combined.head()
df_list[1]

In [ ]:
locid = pd.read_csv('all_loc_id.csv', dtype = {'LocID Root': 'object', 'LocID': 'object'})
locid.head()

In [ ]:
l = ['Ancillary data', 'VBB Seismometer Channels', 'APSS Channels Index', 'SP Seismometer Channels', 'SEIS Software Synthesized Data' ]
locid = locid.drop([0] + locid.index[locid['Channel'].isin(l)].tolist())
locid['LocID Root'].astype('object')
locid.ffill(inplace=True)
locid.rename(columns={'Unnamed: 3' : 'Band'}, inplace=True)
locid.head() 

In [ ]:
locid.to_csv('all_locations_ids.csv', index_label=False)

In [ ]:
# List to hold features for all files (initially empty if not starting fresh)
features_list = []
folder_path = r'C:\Users\ahmed\OneDrive\Desktop\Cosmic Analysts\continuous_waveform'
# Loop over folder and extract features
'''
for filename in os.listdir(folder_path):
    if filename.endswith(".mseed"):
        file_path = os.path.join(folder_path, filename)
        stream = read(file_path)
            
        # Extract features for each trace in the file
        for tr in stream:
            features = extract_features(tr, filename)
            features_list.append(features)
#features_df = pd.DataFrame(features_list)
'''
features_df = pd.read_csv('Mars_InSight_Lander.csv', index_col=False)

# Data Exploration

In [ ]:
features_df.head()

In [ ]:
features_df.drop(columns=['start_time','end_time']).nunique()

In [ ]:
# Convert obspy's UTCDateTime objects to Python datetime objects
features_df['start_time'] = features_df['start_time'].apply(lambda x: x.datetime if isinstance(x, UTCDateTime) else x)
features_df['end_time'] = features_df['end_time'].apply(lambda x: x.datetime if isinstance(x, UTCDateTime) else x)
features_df['start_time'] = pd.to_datetime(features_df['start_time'])
features_df['end_time'] = pd.to_datetime(features_df['end_time'])
features_df['year'] = features_df['start_time'].dt.year
features_df['duration_seconds'] = (features_df['end_time'] - features_df['start_time']).dt.total_seconds()
features_df['start_hour'] = features_df['start_time'].dt.hour
features_df['end_hour'] = features_df['end_time'].dt.hour

Plot features and find whether they are Gaussian or not.
Understand the reason behind some features being NaN.
Add time stamp as a feature to the CSV.

In [ ]:
features_df.info()

In [ ]:
features_df.head()

In [ ]:
features_df['channel'].unique()

In [ ]:
features_df['location'].unique()

In [ ]:
features_df.to_csv('Mars_InSight_Lander.csv', index=False)

In [ ]:
df2 = features_df
loc_cha_to_channel = locid.set_index('locID.CHA')['Channel'].to_dict()

# Map the locID.CHA in df2 to the Channels from df1
df2['Channels'] = df2['locID.CHA'].map(loc_cha_to_channel)

In [ ]:
channel_counts = df2['Channels'].value_counts()
top_channels = channel_counts.head(15)
plt.figure(figsize=(12, 8))
top_channels.plot(kind='bar', edgecolor='black')

plt.xticks(rotation=45, fontsize=10)

plt.title('Top 30 Channels by Frequency')
plt.xlabel('Channels')
plt.ylabel('Frequency')

plt.tight_layout() 
plt.show()


# Feature Engineering using Fast Fourier Transform

In [ ]:
channels_for_feature_engineering = df2[df2['channel'].str[1].isin(['H', 'L'])]
channels_for_feature_engineering


In [ ]:
channels_for_feature_engineering['channel'].nunique()

In [ ]:
'''
engineered_features_list = []
folder_path = r'C:\Users\ahmed\OneDrive\Desktop\Cosmic Analysts\continuous_waveform'
for i in channels_for_feature_engineering['file_name']:
    file_path = os.path.join(folder_path, i)
    stream = read(file_path)
    #print(stream)
    for tr in stream:
        if tr.stats.channel in channels_for_feature_engineering['channel'].unique():
            #print(tr.stats.channel)
            features = feature_engineering(tr, i)
            engineered_features_list.append(features)
        else:
            print(f'Trace{tr} of file {i} not included')
engineered_features = pd.DataFrame(engineered_features_list)
print(engineered_features)
print(engineered_features['channel'].unique())
# Convert obspy's UTCDateTime objects to Python datetime objects
engineered_features['start_time'] = engineered_features['start_time'].apply(lambda x: x.datetime if isinstance(x, UTCDateTime) else x)
engineered_features['end_time'] = engineered_features['end_time'].apply(lambda x: x.datetime if isinstance(x, UTCDateTime) else x)
engineered_features['start_time'] = pd.to_datetime(engineered_features['start_time'])
engineered_features['end_time'] = pd.to_datetime(engineered_features['end_time'])
engineered_features['year'] = engineered_features['start_time'].dt.year
engineered_features['duration_seconds'] = (engineered_features['end_time'] - engineered_features['start_time']).dt.total_seconds()
engineered_features['start_hour'] = engineered_features['start_time'].dt.hour
engineered_features['end_hour'] = engineered_features['end_time'].dt.hour
engineered_features.to_csv('engineered_features_of_velocity_seismometer.csv', index=False)
'''

In [ ]:
engineered_features = pd.read_csv('engineered_features_of_velocity_seismometer.csv', index_col=False)
engineered_features['start_time'] = engineered_features['start_time'].apply(lambda x: x.datetime if isinstance(x, UTCDateTime) else x)
engineered_features['end_time'] = engineered_features['end_time'].apply(lambda x: x.datetime if isinstance(x, UTCDateTime) else x)
engineered_features['start_time'] = pd.to_datetime(engineered_features['start_time'])
engineered_features['end_time'] = pd.to_datetime(engineered_features['end_time'])

engineered_features

# Plotting various graphs based on different columns

In [ ]:
engineered_features.info()

In [ ]:
engineered_features[engineered_features['skewness'].isnull()]

85.LLZ is a transmitted processed data. ESTASP stands for Enhanced Seismic Transient Analysis for Surface Processes, a technique related to seismic data analysis in the context of planetary exploration. It was used in NASA's InSight mission to better understand seismic activities and surface processes on Mars.

In [ ]:
engineered_features.dropna(inplace=True)

In [ ]:
numerical_columns = engineered_features.select_dtypes(include=['float32', 'int32', 'float64', 'float32']).columns

# Plot histograms for each numerical column
engineered_features[numerical_columns].hist(figsize=(12, 10), bins=30, edgecolor='black')

plt.tight_layout()
plt.show()

## 1. Sampling Rate Distribution

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(engineered_features['sampling_rate'], bins=50, kde=True)
plt.title('Distribution of Sampling Rates')
plt.xlabel('Sampling Rate')
plt.ylabel('Frequency')
plt.show()

## 2. Scatter Plot: Mean vs Standard Deviation (std)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(engineered_features['mean'], engineered_features['std'], alpha=0.5)
plt.title('Mean vs. Standard Deviation (std)')
plt.xlabel('Mean')
plt.ylabel('Standard Deviation (std)')
plt.show()

## 3. Correlation Heatmap

In [ ]:
plt.figure(figsize=(24, 16))
corr = engineered_features[numerical_columns].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap of Features')
plt.show()

## 4. Boxplot of energy by Location

In [ ]:
plt.figure(figsize=(21, 9))
sns.boxplot(x='location', y='energy', data=engineered_features)
plt.title('Boxplot of Energy by Location')
plt.xlabel('Location')
plt.ylabel('Energy')
plt.show()

## 5. Line Plot: Duration over time

In [ ]:
plt.figure(figsize=(20, 12))
engineered_features.set_index('start_time')['duration_seconds'].resample('D').mean().plot()
plt.title('Average Duration over Time (Daily)')
plt.ylabel('Duration (Seconds)')
plt.xlabel('Time')
plt.show()

## 6. Pairplot of Selected Features


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(engineered_features['fft_mean'], engineered_features['fft_std'], alpha=0.5)
plt.title('fft_mean vs. fft_std')
plt.xlabel('fft_mean')
plt.ylabel('fft_std')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(engineered_features['skewness'], engineered_features['kurtosis'], alpha=0.5)
plt.title('skewness vs. kurtosis')
plt.xlabel('skewness')
plt.ylabel('kurtosis')
plt.show()

In [ ]:
#'mean', 'std', 'skewness', 'kurtosis', 
sns.pairplot(engineered_features[['mean', 'std', 'skewness']])
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(engineered_features['duration_seconds'], engineered_features['energy'], alpha=0.5)
plt.title('duration_seconds vs. energy')
plt.xlabel('duration_seconds')
plt.ylabel('energy')
plt.show()

## 7. Bar Plot of Start Hour Counts

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='start_hour', data=engineered_features)
plt.title('Count of Records by Start Hour')
plt.xlabel('Start Hour')
plt.ylabel('Count')
plt.show()

## 8. Violin Plot of FFT Std by Station

In [ ]:
plt.figure(figsize=(20, 6))
sns.violinplot(x='station', y='fft_std', data=engineered_features)
plt.title('Distribution of FFT Std by Station')
plt.xlabel('Station')
plt.ylabel('FFT Std')
plt.show()


## 9. Histogram of Skewness and Kurtosis

In [ ]:

plt.figure(figsize=(20, 6))
plt.hist([engineered_features['skewness'].dropna(), engineered_features['kurtosis'].dropna()], bins=50, label=['Skewness', 'Kurtosis'])
plt.title('Histogram of Skewness and Kurtosis')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## 10. Joint Plot: Energy vs Duration

In [ ]:
#sns.jointplot(x='energy', y='duration_seconds', data=features_df, kind='hex', gridsize=30)
#plt.show()

# Dataset and DataLoader initialization for Autoencoder

In [ ]:
cols_to_use = ['locID.CHA','station', 'channel', 'location' , 'sampling_rate', 'num_of_samples',
                'mean', 'std', 'skewness', 'kurtosis', 'fft_mean', 'fft_std', 'energy', 'year',
                  'duration_seconds', 'start_hour', 'end_hour']
cols_to_encode = ['locID.CHA','station', 'channel', 'location' , 'sampling_rate', 'year']
le = LabelEncoder()
for i in cols_to_encode:
    engineered_features[i] = le.fit_transform(engineered_features[i])
# Preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(engineered_features[cols_to_use].values)
X_train, X_temp = train_test_split(X_scaled, test_size=0.15, random_state=seed, shuffle=True)
X_test, X_val = train_test_split(X_temp, test_size=1/3, random_state=seed, shuffle=False)

# Convert the feature data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
# DataLoader for batch processing
train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, X_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataset = TensorDataset(X_test_tensor, X_test_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)


#X_train_tensor = torch.tensor(engineered_features[cols_to_use].values)
#train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
#train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)

In [ ]:
# Initialize the model
model = Autoencoder(X_train_tensor.shape[1])
print(summary(model))
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
train_results = train_autoencoder(model, train_dataloader, val_dataloader, num_epochs=10, num_eval_epoch=1, patience=10 ,optimizer=optimizer, scheduler=scheduler, save_dir=r"C:\Users\ahmed\OneDrive\Desktop\Cosmic Analysts\checkpoints")
reconstruction_errors = train_results['val_reconstruction_errors']
reconstruction_errors

In [ ]:
test_results = evaluate_autoencoder(model, test_dataloader,criterion=nn.MSELoss(), device='cpu')
test_reconstruction_errors = test_results["val_reconstruction_error"]

In [ ]:
#file_path = r'C:\Users\ahmed\OneDrive\Desktop\Cosmic Analysts\XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed'
#process_seismic_data(model, file_path, threshold)

In [ ]:

# Plot loss over epochs
plt.figure(figsize=(20, 6))

# Plot training and validation loss over epochs
plt.subplot(1, 3, 1)
epochs_range = range(1, len(train_results["train_loss"]) + 1)
plt.plot(epochs_range, train_results["train_loss"], label='Train Loss', marker='o')
plt.plot(epochs_range, train_results["val_loss"], label='Validation Loss', marker='x')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Validation Loss Over Epochs')
plt.legend()

# Plot validation reconstruction error with threshold
plt.subplot(1, 3, 2)
val_reconstruction_errors_flat = [item for sublist in train_results["val_reconstruction_errors"] for item in sublist]
threshold = np.percentile(val_reconstruction_errors_flat, 99)  

plt.hist(val_reconstruction_errors_flat, bins=50, alpha=0.7, label='Validation Reconstruction Error')
plt.axvline(threshold, color='r', linestyle='dashed', linewidth=1, label='Threshold (95th Percentile)')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.title('Validation Error Distribution')
plt.legend()

# Plot test reconstruction error with threshold
plt.subplot(1, 3, 3)
plt.hist(test_reconstruction_errors, bins=50, alpha=0.7, label='Test Reconstruction Error')
plt.axvline(threshold, color='r', linestyle='dashed', linewidth=1, label='Threshold (95th Percentile)')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.title('Test Error Distribution')
plt.legend()

# Plot validation anomalies count over epochs
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_results["val_anomalies_counts"]) + 1), train_results["val_anomalies_counts"], marker='o', color='b', label='Validation Anomalies Count')
plt.xlabel('Epoch')
plt.ylabel('Number of Anomalies')
plt.title('Validation Anomalies Count Over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
train_results["val_anomalies_counts"]

In [ ]:
test_results["val_anomalies_count"]

# Unsupervised ML

## Clustering

In [ ]:
df = pd.read_csv('engineered_features_of_velocity_seismometer.csv', index_col=False)
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])
cols_to_use = ['locID.CHA','station', 'channel', 'location' , 'sampling_rate', 'num_of_samples',
                'mean', 'std', 'skewness', 'kurtosis', 'fft_mean', 'fft_std', 'energy', 'year',
                  'duration_seconds', 'start_hour', 'end_hour']
cols_to_encode = ['locID.CHA','station', 'channel', 'location' , 'sampling_rate', 'year']
le = LabelEncoder()
for i in cols_to_encode:
    df[i] = le.fit_transform(df[i])
print(df.info())

In [ ]:
df.dropna(inplace=True)

In [ ]:
data = df[cols_to_use]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
# reduce to 2 importants features
pca = PCA(n_components=2)
data = pca.fit_transform(data)
# standardize these 2 new features
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)

In [ ]:
# calculate with different number of centroids to see the loss plot (elbow method)
n_cluster = range(1, 20)
kmeans = [KMeans(n_clusters=i).fit(data) for i in n_cluster]
scores = [kmeans[i].score(data) for i in range(len(kmeans))]
fig, ax = plt.subplots()
ax.plot(n_cluster, scores)
plt.show()

In [ ]:
# I choose 14 centroids arbitrarily and add these data to the central dataframe
df['cluster'] = kmeans[14].predict(data)
df['principal_feature1'] = data[0]
df['principal_feature2'] = data[1]
df['cluster'].value_counts()

In [ ]:
#plot the different clusters with the 2 main features
fig, ax = plt.subplots()
colors = {0:'red', 1:'blue', 2:'green', 3:'pink', 4:'black', 5:'orange', 6:'cyan', 7:'yellow', 8:'brown', 9:'purple', 10:'white', 11: 'grey', 12:'lightblue', 13:'lightgreen', 14: 'darkgrey'}
ax.scatter(df['principal_feature1'], df['principal_feature2'], c=df["cluster"].apply(lambda x: colors[x]))
plt.show()

In [ ]:
# get the distance between each point and its nearest centroid. The biggest distances are considered as anomaly
def getDistanceByPoint(data, model):
    distance = pd.Series()
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.at[i] = np.linalg.norm(Xa - Xb)
    return distance
outliers_fraction = 0.01
distance = getDistanceByPoint(data, kmeans[14])
number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
# anomaly21 contain the anomaly result of method 2.1 Cluster (0:normal, 1:anomaly) 
df['anomaly21'] = (distance >= threshold).astype(int)
print(df['anomaly21'].value_counts())

In [ ]:
import matplotlib.dates as mdates
# Select anomaly data
a = df.loc[df['anomaly21'] == 1, ['start_time', 'duration_seconds']]

fig, ax = plt.subplots(figsize=(30, 6))

# Convert datetime64 to numerical format using mdates.date2num
a['start_time_num'] = mdates.date2num(a['start_time'])

# Resample and plot the average duration per day
df.set_index('start_time')['duration_seconds'].resample('D').mean().plot(ax=ax)

# Plot anomalies using the converted numerical dates
ax.scatter(a['start_time_num'], a['duration_seconds'], color='red')

# Simplify date formatting on x-axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=6))  # Show labels only for every nth day (change interval as needed)

# Rotate date labels for better visibility
plt.xticks(rotation=45)

# Add x-axis label instead of individual dates
plt.xlabel('Date')

plt.show()



## Isolation Forest

In [ ]:
data = df[cols_to_use]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
# train isolation forest 
model =  IsolationForest(contamination = outliers_fraction)
model.fit(data)
# add the data to the main  
df['anomaly25'] = pd.Series(model.predict(data))
df['anomaly25'] = df['anomaly25'].map( {1: 0, -1: 1} )
print(df['anomaly25'].value_counts())

In [ ]:
a = df.loc[df['anomaly25'] == 1, ['start_time', 'duration_seconds']]
# Convert datetime64 to numerical format using mdates.date2num
a['start_time_num'] = mdates.date2num(a['start_time'])

fig, ax = plt.subplots(figsize=(30, 6))

# Convert datetime64 to numerical format using mdates.date2num
a['start_time_num'] = mdates.date2num(a['start_time'])

# Resample and plot the average duration per day
df.set_index('start_time')['duration_seconds'].resample('D').mean().plot(ax=ax)

# Plot anomalies using the converted numerical dates
ax.scatter(a['start_time_num'], a['duration_seconds'], color='red')

# Simplify date formatting on x-axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=6))  # Show labels only for every nth day (change interval as needed)

# Rotate date labels for better visibility
plt.xticks(rotation=45)

# Add x-axis label instead of individual dates
plt.xlabel('Date')

plt.show()

In [ ]:
file_path = r'C:\Users\ahmed\OneDrive\Desktop\Cosmic Analysts\XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed'
st = read(file_path)
tr = st[0]  # Assuming only one trace for simplicity

# Extract data and timestamps
tr_data = tr.data
tr_times = np.linspace(0, len(tr_data) / tr.stats.sampling_rate, num=len(tr_data))

plt.figure(figsize=(12, 4))
plt.plot(tr_times, tr_data)
plt.title('Seismic Trace')
plt.xlabel('Time (s)')
plt.ylabel('Velocity')
plt.show()

In [ ]:
# Normalize the data (optional)
tr_data_norm = (tr_data - np.mean(tr_data)) / np.std(tr_data)

# Reshape the data for isolation forest (assuming each sample is a separate point)
data_points = tr_data_norm.reshape(-1, 1)

In [ ]:
# Initialize Isolation Forest
iso_forest = IsolationForest(contamination=0.01, random_state=42)

# Fit the model on seismic data
iso_forest.fit(data_points)

# Predict anomalies
anomalies = iso_forest.predict(data_points)

# Label anomalies as -1 and normal data as 1
anomalies_indices = np.where(anomalies == -1)[0]


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(tr_times, tr_data_norm, label='Seismic Data')
plt.scatter(tr_times[anomalies_indices], tr_data_norm[anomalies_indices], color='red', label='Detected Events', marker='x')
plt.title('Seismic Event Detection Using Isolation Forest')
plt.xlabel('Time (s)')
plt.ylabel('Normalized Velocity')
plt.legend()
plt.show()

# Optionally, print the times where anomalies were detected
event_times = tr_times[anomalies_indices]
print("Seismic events detected at times (s):", event_times)

In [ ]:
file_path = r'C:\Users\ahmed\OneDrive\Desktop\Cosmic Analysts\XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed'
st = read(file_path)
tr = st[0]  # Assuming only one trace for simplicity

# Extract data and timestamps
tr_data = tr.data
tr_times = np.linspace(0, len(tr_data) / tr.stats.sampling_rate, num=len(tr_data))

# Normalize the data (optional)
tr_data_norm = (tr_data - np.mean(tr_data)) / np.std(tr_data)

# Reshape the data for isolation forest (assuming each sample is a separate point)
data_points = tr_data_norm.reshape(-1, 1)

# Initialize Isolation Forest
iso_forest = IsolationForest(contamination=0.0007, random_state=42)

# Fit the model on seismic data
iso_forest.fit(data_points)

# Predict anomalies
anomalies = iso_forest.predict(data_points)

# Label anomalies as -1 and normal data as 1
anomalies_indices = np.where(anomalies == -1)[0]

# Threshold for event separation (in seconds)
min_separation_time = 1  # adjust this value based on your data's timescale
last_event_time = -np.inf  # initialize to negative infinity

filtered_anomalies_indices = []
for idx in anomalies_indices:
    if tr_times[idx] - last_event_time > min_separation_time:
        filtered_anomalies_indices.append(idx)
        last_event_time = tr_times[idx]  # update the last event time

filtered_anomalies_indices = np.array(filtered_anomalies_indices)

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(tr_times, tr_data_norm, label='Seismic Data')
plt.scatter(tr_times[filtered_anomalies_indices], tr_data_norm[filtered_anomalies_indices], color='red', label='Detected Events', marker='x')
plt.title('Seismic Event Detection with Time-Based Separation')
plt.xlabel('Time (s)')
plt.ylabel('Normalized Velocity')
plt.legend()
plt.show()

# Optionally, print the times where filtered anomalies were detected
event_times = tr_times[filtered_anomalies_indices]
print("Filtered seismic events detected at times (s):", event_times)

In [ ]:
from obspy.signal.invsim import cosine_taper
from obspy.signal.filter import highpass
from obspy.signal.trigger import classic_sta_lta, plot_trigger, trigger_onset

# Sampling frequency of our trace
df = tr.stats.sampling_rate

# How long should the short-term and long-term window be, in seconds?
sta_len = 120
lta_len = 600

# Run Obspy's STA/LTA to obtain a characteristic function
# This function basically calculates the ratio of amplitude between the short-term 
# and long-term windows, moving consecutively in time across the data
cft = classic_sta_lta(tr_data, int(sta_len * df), int(lta_len * df))

# Plot characteristic function
fig,ax = plt.subplots(1,1,figsize=(12,3))
ax.plot(tr_times,cft)
ax.set_xlim([min(tr_times),max(tr_times)])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Characteristic function')

In [ ]:
# Play around with the on and off triggers, based on values in the characteristic function
thr_on = 4
thr_off = 1.5
on_off = np.array(trigger_onset(cft, thr_on, thr_off))
# The first column contains the indices where the trigger is turned "on". 
# The second column contains the indices where the trigger is turned "off".

# Plot on and off triggers
fig,ax = plt.subplots(1,1,figsize=(15,6))
for i in np.arange(0,len(on_off)):
    triggers = on_off[i]
    ax.axvline(x = tr_times[triggers[0]], color='red', label='Trig. On')
    print(f'Event detected at {tr_times[triggers[0]]}')
    ax.axvline(x = tr_times[triggers[1]], color='purple', label='Trig. Off')

# Plot seismogram
ax.plot(tr_times,tr_data)
ax.set_xlim([min(tr_times),max(tr_times)])
ax.legend()

## One Class SVM

In [ ]:
df = pd.read_csv('engineered_features_of_velocity_seismometer.csv', index_col=False)
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])
cols_to_use = ['locID.CHA','station', 'channel', 'location' , 'sampling_rate', 'num_of_samples',
                'mean', 'std', 'skewness', 'kurtosis', 'fft_mean', 'fft_std', 'energy', 'year',
                  'duration_seconds', 'start_hour', 'end_hour']
cols_to_encode = ['locID.CHA','station', 'channel', 'location' , 'sampling_rate', 'year']
le = LabelEncoder()
for i in cols_to_encode:
    df[i] = le.fit_transform(df[i])
df.dropna(inplace=True)
data = df[cols_to_use]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
# train one class SVM 
model =  OneClassSVM(nu=0.95 * outliers_fraction) #nu=0.95 * outliers_fraction  + 0.05
data = pd.DataFrame(np_scaled)
model.fit(data)
# add the data to the main  
df['anomaly26'] = pd.Series(model.predict(data))
df['anomaly26'] = df['anomaly26'].map( {1: 0, -1: 1} )
print(df['anomaly26'].value_counts())

In [ ]:
a = df.loc[df['anomaly26'] == 1, ['start_time', 'duration_seconds']]
# Convert datetime64 to numerical format using mdates.date2num
a['start_time_num'] = mdates.date2num(a['start_time'])

fig, ax = plt.subplots(figsize=(30,6))

# Convert datetime64 to numerical format using mdates.date2num
a['start_time_num'] = mdates.date2num(a['start_time'])

# Resample and plot the average duration per day
df.set_index('start_time')['duration_seconds'].resample('D').mean().plot(ax=ax)

# Plot anomalies using the converted numerical dates
ax.scatter(a['start_time_num'], a['duration_seconds'], color='red')

# Simplify date formatting on x-axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=6))  # Show labels only for every nth day (change interval as needed)

# Rotate date labels for better visibility
plt.xticks(rotation=45)

# Add x-axis label instead of individual dates
plt.xlabel('Date')

plt.show()